In [1]:
import pandas as pd                                                                                  
                                                                                                     
# Read in data                                                                                       
data = pd.read_csv(                                                                                  
    "auto-mpg.data", delim_whitespace=True, na_values="?", header=None,                              
    names=["mpg", "cylinders", "displacement", "horsepower", "weight",                               
           "acceleration", "model year", "origin", "car name", ])                                    
# Drop string feature                                                                                
data.pop("car name")                                                                                 
# Replace N/A values in horsepower                                                                   
data["horsepower"].fillna(data["horsepower"].median(), inplace=True)                                 
# Separate target feature                                                                            
target = data.pop("mpg")

In [2]:
data.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,8,307.0,130.0,3504.0,12.0,70,1
1,8,350.0,165.0,3693.0,11.5,70,1
2,8,318.0,150.0,3436.0,11.0,70,1
3,8,304.0,150.0,3433.0,12.0,70,1
4,8,302.0,140.0,3449.0,10.5,70,1


In [3]:
target

0      18.0
1      15.0
2      18.0
3      16.0
4      17.0
       ... 
393    27.0
394    44.0
395    32.0
396    28.0
397    31.0
Name: mpg, Length: 398, dtype: float64

In [4]:
3504.0/307

11.413680781758957

In [5]:
data.origin.value_counts()

1    249
3     79
2     70
Name: origin, dtype: int64

### Operator

In [6]:
import operator                                                                                      
                                                                                                     
val1 = {"feature_name": "horsepower"}                                                                
val2 = {"feature_name": "cylinders"}                                                                 
val3 = {"feature_name": "weight"}                                                                    
node1 = {                                                                                            
    "func": operator.add,                                                                            
    "children": [val1, val2],                                                                        
    "format_str": "({} + {})",                                                                       
}                                                                                                    
program = {                                                                                          
    "func": operator.mul,                                                                            
    "children": [node1, val3],                                                                       
    "format_str": "({} * {})",                                                                       
}

In [7]:
def render_prog(node):                                                                               
    if "children" not in node:                                                                       
        return node["feature_name"]                                                                  
    return node["format_str"].format(*[render_prog(c) for c in node["children"]])  

In [8]:
print(render_prog(program))

((horsepower + cylinders) * weight)


In [9]:
def safe_div(a, b):                                                                                  
    return a / b if b else a                                                                         



operations = (                                                                                       
    {"func": operator.add, "arg_count": 2, "format_str": "({} + {})"},                               
    {"func": operator.sub, "arg_count": 2, "format_str": "({} - {})"},                               
    {"func": operator.mul, "arg_count": 2, "format_str": "({} * {})"},                               
    {"func": safe_div, "arg_count": 2, "format_str": "({} / {})"},                                   
    {"func": operator.neg, "arg_count": 1, "format_str": "-({})"},    
)

In [10]:
def evaluate(node, row):                                                                             
    if "children" not in node:                                                                       
        return row[node["feature_name"]]                                                             
    return node["func"](*[evaluate(c, row) for c in node["children"]])  

In [11]:
print(evaluate(program, data.iloc[0]))

483552.0


### GP

In [12]:
from random import randint, random, seed                                                             
                                                                                                     
seed(0)                                                                                              
                                                                                                     
def random_prog(depth):                                                                              
    # favor adding function nodes near the tree root and 
    # leaf nodes as depth increases                           
    if randint(0, 10) >= depth * 2:                                                                  
        op = operations[randint(0, len(operations) - 1)]                                             
        return {                                                                                     
            "func": op["func"],                                                                      
            "children": [random_prog(depth + 1) for _ in range(op["arg_count"])],                    
            "format_str": op["format_str"],                                                          
        }                                                                                            
    else:                                                                                            
        return {"feature_name": data.columns[randint(0, data.shape[1] - 1)]}                         
                                                                                                     
                                                                                                     
POP_SIZE = 300                                                                                       
population = [random_prog(0) for _ in range(POP_SIZE)] 

In [13]:
print(render_prog(population[0]))   

(horsepower / ((((displacement - displacement) * acceleration) * acceleration) / (cylinders - origin)))


In [14]:
def select_random_node(selected, parent, depth):                                                     
    if "children" not in selected:                                                                   
        return parent                                                                                
    # favor nodes near the root                                                                      
    if randint(0, 10) < 2*depth:                                                                     
        return selected                                                                              
    child_count = len(selected["children"])                                                          
    return select_random_node(
        selected["children"][randint(0, child_count - 1)], 
        selected, depth+1) 

In [15]:
print(render_prog(select_random_node(program, None, 0)))

((horsepower + cylinders) * weight)


In [16]:
from copy import deepcopy
 
def do_mutate(selected):
    offspring = deepcopy(selected)
    mutate_point = select_random_node(offspring, None, 0)
    child_count = len(mutate_point["children"])
    mutate_point["children"][randint(0, child_count - 1)] = random_prog(0)
    return offspring

In [17]:
print(render_prog(do_mutate(program)))

(((-(displacement) - (weight / (origin / weight))) + cylinders) * weight)


In [18]:
def do_xover(selected1, selected2):                                                                  
    offspring = deepcopy(selected1)                                                                  
    xover_point1 = select_random_node(offspring, None, 0)                                            
    xover_point2 = select_random_node(selected2, None, 0)                                            
    child_count = len(xover_point1["children"])                                                      
    xover_point1["children"][randint(0, child_count - 1)] = xover_point2                             
    return offspring       

In [19]:
print(render_prog(do_xover(population[0], population[1])))

(horsepower / (((-(horsepower) * acceleration) * acceleration) / (cylinders - origin)))


In [20]:
TOURNAMENT_SIZE = 3                                                                                  
                        
def get_random_parent(population, fitness):                                                          
    # randomly select population members for the tournament                                          
    tournament_members = [
        randint(0, POP_SIZE - 1) for _ in range(TOURNAMENT_SIZE)]                  
    # select tournament member with best fitness                                                     
    member_fitness = [(fitness[i], population[i]) for i in tournament_members]                       
    return min(member_fitness, key=lambda x: x[0])[1]

In [21]:
XOVER_PCT = 0.7                                                                                                                                                            
                                                                                                     
def get_offspring(population, fitness):                                                              
    parent1 = get_random_parent(population, fitness)                                                 
    if random() > XOVER_PCT:                                                                         
        parent2 = get_random_parent(population, fitness)                                             
        return do_xover(parent1, parent2)                                                            
    else:                                                                                            
        return do_mutate(parent1) 

In [22]:
REG_STRENGTH = 0.5                                                                                   

def node_count(x):                                                                                   
    if "children" not in x:                                                                          
        return 1                                                                                     
    return sum([node_count(c) for c in x["children"]]) 

def compute_fitness(program, prediction):                                                            
    mse = ((pd.Series(prediction) - target) ** 2).mean()                                             
    penalty = node_count(program) ** REG_STRENGTH                                                    
    return mse * penalty  

In [23]:
MAX_GENERATIONS = 10                                                                                 
                                                                                                     
global_best = float("inf")                                                                           
for gen in range(MAX_GENERATIONS):                                                                   
    fitness = []                                                                                     
    for prog in population:                                                                          
        prediction = [                                                                               
            evaluate(prog, row) for _, row in data.iterrows()]                                       
        score = compute_fitness(prog, prediction)                                                    
        fitness.append(score)                                                                        
        if score < global_best:                                                                      
            global_best = score                                                                      
            best_pred = prediction                                                                   
            best_prog = prog                                                                         
    print(                                                                                           
        "Generation: %d\nBest Score: %.2f\nMedian score: %.2f\nBest program: %s\n"                   
        % (                                                                                          
            gen,                                                                                     
            global_best,                                                                             
            pd.Series(fitness).median(),                                                             
            render_prog(best_prog),                                                                  
        )                                                                                            
    )                                                                                                
    population = [                                                                                   
        get_offspring(population, fitness)                                                           
        for _ in range(POP_SIZE)]                                                                                                             


Generation: 0
Best Score: 106.60
Median score: 439225.90
Best program: (cylinders - -(acceleration))

Generation: 1
Best Score: 86.05
Median score: 4393.05
Best program: ((origin * cylinders) + acceleration)

Generation: 2
Best Score: 84.32
Median score: 2531.16
Best program: (origin - -((weight / displacement)))

Generation: 3
Best Score: 84.32
Median score: 2377.53
Best program: (origin - -((weight / displacement)))

Generation: 4
Best Score: 84.32
Median score: 2213.35
Best program: (origin - -((weight / displacement)))

Generation: 5
Best Score: 84.32
Median score: 2295.95
Best program: (origin - -((weight / displacement)))

Generation: 6
Best Score: 84.32
Median score: 2525.15
Best program: (origin - -((weight / displacement)))

Generation: 7
Best Score: 84.32
Median score: 2364.86
Best program: (origin - -((weight / displacement)))

Generation: 8
Best Score: 84.32
Median score: 2518.05
Best program: (origin - -((weight / displacement)))

Generation: 9
Best Score: 84.32
Median sco

In [24]:
print("Best score: %f" % global_best)                                                                
print("Best program: %s" % render_prog(best_prog))                                                   
output = {"target": target, "pred": best_pred}                                                       
pd.DataFrame(output).to_csv("best_pred.csv")  

Best score: 84.320464
Best program: (origin - -((weight / displacement)))
